# Kitaev Model: Toric Code

Kitaev Hamiltonian:
$
\begin{equation}
H_K = - \sum_p B_p - \sum_s A_s
\end{equation}
$

With operators defined as:

$
\begin{align}
B_p &= \Pi_{(ij) \in p} \sigma_{(ij)}^z \\
A_s &= \Pi_{(ij) \in s} \sigma_{(ij)}^x \\
\end{align}
$

where $p$ is a plaquette of the lattice and $s$ is a star vertex of the lattice.

In [3]:
import numpy as np

Consider an $N=3$ torus, meaning there are $N^2 = 9$ lattice sites